In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from Bio import pairwise2

/home/dmilone/anaconda3/lib/python3.7/site-packages/Bio/pairwise2.py:283: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  BiopythonDeprecationWarning,
/home/dmilone/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data

In [ ]:
DATA_PATH = "/DATA/lncRNA/data/"

In [ ]:
raw_fn = "archiveII_220808"
dref_all = pd.read_csv(DATA_PATH + raw_fn + ".csv", index_col = "id")
#display(dref_all.head(3))
display(dref_all.shape)

#todo sort by length <<
#todo split in slices to run in serveral CPUs

(3975, 10)

# Diccionario con sustituciones

Valores por default de BLAST (Emilio):

- Match = 1
- Mismatch = -2
- GapOpen = -5
- GapExtend = -2

In [ ]:
Match = 1
Mismatch = -2
GapOpen=-5
GapExtend=-2

simbolos=["A","C","G","U"]

match_dic = {}
for simbolA in simbolos:
    for simbolB in simbolos:
        if simbolA==simbolB:
            match_dic[(simbolA, simbolB)] = Match
        else:
            match_dic[(simbolA, simbolB)] = Mismatch

# Bucles de comparacion
Todos contra todos


In [ ]:
#dref = dref_all.head(10) # testing
dref = dref_all

sims = pd.DataFrame(np.zeros((len(dref.index),len(dref.index))))
sims.index = dref.index
sims.columns = dref.index
for i, row in enumerate(tqdm(dref.index)):
    row_seq = dref.loc[row].sequence
    for j, col in enumerate(dref.index):
        if i<j:
            col_seq = dref.loc[col].sequence
            alignm = pairwise2.align.localds(col_seq, row_seq, match_dic, GapOpen, GapExtend, one_alignment_only=True)[0]
            max_len = max(len(alignm.seqB),len(alignm.seqA))
            IDscore = sum(a==b for a,b in zip(alignm.seqA, alignm.seqB)) / max_len
            sims.loc[row,col] = IDscore
        elif i>j:
            sims.loc[row,col] = sims.loc[col,row]
        else:
            sims.loc[row,col] = 1.0

100%|██████████| 3975/3975 [54:31:24<00:00, 49.38s/it]     


In [ ]:
#sims.to_csv(DATA_PATH + "seqsim_f_all.csv")
sims.to_hdf(DATA_PATH + "seqsim_f_all.h5", key='rnadist', mode='w')